In [2]:
# Required packages:
# pip3 install requests pandas
import requests
import json
from enum import Enum
import pandas as pd
import math

def unpack(s): return list(s) if type(s) != 'list' else s
def head(s): return s[0] if len(s) > 0 else []
def tail(s): return s[1:]
def reverse(s): return s[::-1]
def last(s): return s[-1:]
def init(s): return s[0:len(s)-1]
def take(s, n): return s[:n]
def drop(s, n): return s[n:]
def product(s): return reduce(lambda x, y: x * y, s)

class TextType(Enum):
    Description = 0
    ExtendedDescription = 1
    MoreInfo = 2

class ElfskotApi():

    base_address = 'https://api.elfskot.cloud/api/2/'

    def __init__(self, application_id, secret):
        self.application_id = application_id
        self.secret = secret
        self.get_token()

    def get_token(self):
        payload = { 'clientId': self.application_id, 'secret': self.secret}
        result = requests.post(self.get_url('auth/elfskotconnectlogin'), json=payload)
        self.check_result(result)
        self.token = json.loads(result.text)['accessToken']

    def check_result(self, result):
        if result.status_code != 200:
            raise ValueError('Error: API returned status code {}'.format(result.status_code))
        return True

    def get_auth_header(self): return {'Authorization': 'bearer {}'.format(self.token)}
    def get_url(self, endpoint): return self.base_address + endpoint

    def http_return_if_valid(self, result):
        self.check_result(result)
        return json.loads(result.text)

    def http_get(self, endpoint):
        return self.http_return_if_valid(
            requests.get(self.get_url(endpoint), headers=self.get_auth_header())
        )

    def http_post(self, endpoint, o):
        return self.http_return_if_valid(
            requests.post(self.get_url(endpoint), json=o, headers=self.get_auth_header())
        )

    def http_put(self, endpoint, o):
        return self.http_return_if_valid(
            requests.put(self.get_url(endpoint), json=o, headers=self.get_auth_header())
        )

    def http_delete(self, endpoint, key):
        result = requests.delete(self.get_url(endpoint) + '/{}'.format(key), headers=self.get_auth_header())
        if result.status_code != 200: 
            raise ValueError('Error: API returned status code {}'.format(result.status_code))

    def query(self, endpoint): return Query(endpoint, self.http_get)
    def all(self, endpoint): return self.query(endpoint)
    def find(self, endpoint, p, v): return self.query(endpoint).filter(p, v)
    def get(self, endpoint, k): return self.find(endpoint, 'id', k)
    def new(self, endpoint, o): return self.http_post(endpoint, o)
    def update(self, endpoint, o): return self.http_put(endpoint, o)
    def delete(self, endpoint, k): self.http_delete(endpoint, k)  
    def help(self, endpoint): print('Model for {}:\r\n{}'
                                    .format(endpoint, list(self.first(endpoint, 'id', '').keys())))
        
class Query():
    
    def __init__(self, endpoint, http):
        self.http = http
        self.endpoint = endpoint
        self.parameters = {}
        self.data = []
        self.index = 0
        
    def raise_(self, t): raise ValueError(t)
    
    def skip(self, n):
        if 'skip' in self.parameters: raise ValueError('Skip already set.')
        self.parameters['skip'] = n
        self.index = n
        return self
    
    def take(self, n):
        if 'limit' in self.parameters: self.raise_('Take already called.')
        self.parameters['limit'] = n
        return self
    
    def include(self, name):
        includes = []
        if 'include' in self.parameters:
            includes = self.parameters['include'].split(',')
        includes.append(name)
        self.parameters['include'] = ",".join(includes)
        return self
    
    def filter(self, property, value):
        self.parameters[property] = value
        return self
    
    def orderBy(self, property, descending = False):
        self.parameters['orderby'] = property
        return self
    
    def descending(self):
        self.parameters['descending'] = True
        return self
    
    def url_qry_params(self):
        return '{}?{}'.format(self.endpoint, '&'.join(['{}={}'.format(k,v) 
                                                       for k,v in dict(self.parameters).items()])).lower()
    
    def __next__(self): 
        if self.data == []: self.data = self.http(self.url_qry_params())
        try: result = self.data[self.index]
        except IndexError: raise StopIteration
        self.index += 1
        return result
    
    def __iter__(self):
        return self
    
    def list(self): return list(self)
    def df(self): return pd.DataFrame(self.list())
    def first(self): return head(self.list())

In [213]:
db = ElfskotApi('5fce47a9-e9d0-48bc-b159-357537a15dce', 'i1ll1aiv')
db.query('features').filter('name', 'S6000').include('texts').df()

ValueError: Error: API returned status code 401

In [188]:
quotations = db.query('quotations').take(5).include('lines').df()

In [189]:
quotations.columns

Index(['createdDate', 'creatorId', 'currency', 'currencyIso', 'customField1',
       'customField2', 'customField3', 'customField4', 'customField5',
       'debtor', 'debtorId', 'deliverydate', 'exchangeRate', 'expiresDate',
       'files', 'formattedCreatedDate', 'formattedDeliveryDate',
       'formattedExpiresDate', 'formattedUpdatedDate', 'id', 'isExpired',
       'languageIso', 'leaseTermAmount', 'leaseTermType', 'lines', 'margin',
       'materialList', 'organization', 'organizationId', 'organizationName',
       'propertyIds', 'quotationNumber', 'quotationTemplate',
       'quotationTemplateId', 'reference', 'remarks', 'revisionOfId', 'seller',
       'sellerId', 'sellerLogo', 'sellerName', 'shipTo', 'shipToId', 'status',
       'statusDescription', 'subTotalExclPrice', 'subTotalExclPriceLabel',
       'subTotalInclPrice', 'subTotalInclPriceLabel', 'subTotalMargin',
       'subTotalMarginLabel', 'subTotalNumberFormatted',
       'subTotalPurchasePrice', 'subTotalPurchasePriceLab

In [1]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

NameError: name 'pd' is not defined

In [191]:
quotations

,createdDate,creatorId,currency,currencyIso,customField1,customField2,customField3,customField4,customField5,debtor,debtorId,deliverydate,exchangeRate,expiresDate,files,formattedCreatedDate,formattedDeliveryDate,formattedExpiresDate,formattedUpdatedDate,id,isExpired,languageIso,leaseTermAmount,leaseTermType,lines,margin,materialList,organization,organizationId,organizationName,propertyIds,quotationNumber,quotationTemplate,quotationTemplateId,reference,remarks,revisionOfId,seller,sellerId,sellerLogo,sellerName,shipTo,shipToId,status,statusDescription,subTotalExclPrice,subTotalExclPriceLabel,subTotalInclPrice,subTotalInclPriceLabel,subTotalMargin,subTotalMarginLabel,subTotalNumberFormatted,subTotalPurchasePrice,subTotalPurchasePriceLabel,subject,synced,totalDiscountAmntLabel,totalDiscountAmt,totalDiscountNumberFormatted,totalDiscountPct,totalDiscountPctNumberFormatted,totalExclPrice,totalInclPrice,totalMargin,totalMarginDiscount,totalMarginDiscountLabel,totalMarginLabel,totalPriceExclVATLabel,totalPriceInclVAT,totalPriceInclVATLabel,totalPriceInclVATNumberFormatted,totalPriceNumberFormatted,totalPurchasePriceLabel,totalVatAmountNumberFormatted,updatedDate,versionNumber,wholeSaleDiscountAmt,wholeSaleDiscountAmtLabel,wholeSaleDiscountAmtNumberFormatted,wholeSaleDiscountMinusAmtLabel,wholeSaleDiscountPct,wholeSaleDiscountRows
0,2018-01-26T14:19:57.3177359+00:00,00000000-0000-0000-0000-000000000000,None,eur,None,None,None,None,None,None,None,None,1.0,0001-01-01T00:00:00+00:00,None,26-01-2018,,01-01-0001,22-02-2018,86ae79aa-dd4a-47e9-a439-08d564c7e0a3,True,nl,0,0,[{'quotationId': '86ae79aa-dd4a-47e9-a439-08d5...,None,None,None,None,None,[],None,None,None,None,None,None,None,None,None,,None,None,5,Expired,0.0,0.00,0.0,0.00,0.0,0.00,"0,00",0.0,0.00,None,False,0.00,0.0,"0,00",0.0,"0,00",0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,"0,00","0,00",0.00,"0,00",2018-02-22T14:16:52.1023463+00:00,0.0,0.0,0.00,"0,00",0.00,0.0,None
1,2018-02-01T09:42:54.2580772+00:00,00000000-0000-0000-0000-000000000000,None,eur,None,None,None,None,None,None,None,None,1.0,0001-01-01T00:00:00+00:00,None,01-02-2018,,01-01-0001,22-02-2018,69e8a4ba-829d-4dda-e659-08d569582b00,False,nl,0,0,[{'quotationId': '69e8a4ba-829d-4dda-e659-08d5...,None,None,None,None,None,[],None,None,None,None,None,None,None,None,None,,None,None,2,Accepted,37480.0,"37,480.00",37480.0,"37,480.00",37480.0,"37,480.00","37.480,00",0.0,0.00,None,False,0.00,0.0,"0,00",0.0,"0,00",37480.0,37480.0,37480.0,0.0,0.00,"37,480.00","37,480.00",37480.0,"37,480.00","37.480,00","37.480,00",0.00,"37.480,00",2018-02-22T13:13:26.0497306+00:00,1.0,0.0,0.00,"0,00",0.00,0.0,None
2,2018-02-02T07:33:45.0978448+00:00,00000000-0000-0000-0000-000000000000,None,eur,None,None,None,None,None,None,None,None,1.0,0001-01-01T00:00:00+00:00,None,02-02-2018,,01-01-0001,05-06-2018,13319212-a881-4d67-b681-08d56a0f4a8c,False,nl,0,0,[{'quotationId': '13319212-a881-4d67-b681-08d5...,None,None,None,None,None,[],None,None,None,None,None,None,None,None,None,,None,None,2,Accepted,35480.0,"35,480.00",35480.0,"35,480.00",35480.0,"35,480.00","35.480,00",0.0,0.00,None,False,0.00,0.0,"0,00",0.0,"0,00",35480.0,35480.0,35480.0,0.0,0.00,"35,480.00","35,480.00",35480.0,"35,480.00","35.480,00","35.480,00",0.00,"35.480,00",2018-06-05T12:47:38.0299299+00:00,1.0,0.0,0.00,"0,00",0.00,0.0,None
3,2018-02-06T17:08:22.117911+00:00,00000000-0000-0000-0000-000000000000,None,eur,None,None,None,None,None,None,None,None,1.0,0001-01-01T00:00:00+00:00,None,06-02-2018,,01-01-0001,12-06-2018,a111e0f9-79a6-4bca-3a88-08d56d843a0c,False,nl,0,0,[{'quotationId': 'a111e0f9-79a6-4bca-3a88-08d5...,None,None,None,None,None,[],None,None,None,None,None,None,None,None,None,,None,None,2,Accepted,77660.0,"77,660.00",77660.0,"77,660.00",77660.0,"77,660.00","77.660,00",0.0,0.00,None,False,0.00,0.0,"0,00",0.0,"0,00",77660.0,77660.0,77660.0,0.0,0.00,"77,660.00","77,660.00",77660.0,"77,660.00","77.660,00","77.660,00",0.00,"77.660,00",2018-06-12T15:13:37.6543896+00:00,NaN,0.0,0.00,"0,00",

In [211]:
def print_quotation(q):
    print('Id: {}'.format(q['id']))
    print('Expired: {}'.format(q['isExpired']))
    
    for l in q['lines']:
        print('{} Item: {}\t Qty: {}\t Price: {}'.format(l['lineNumber'], l['articleCode'], l['quantityAmount'], l['unitPrice']))

q = db.all('quotations').include('lines').take(1).first()
print_quotation(q)

Id: 86ae79aa-dd4a-47e9-a439-08d564c7e0a3
Expired: True
0 Item: X_108	 Qty: 1.0	 Price: 0.0
0 Item: X_107	 Qty: 4.0	 Price: 0.0
0 Item: X_101	 Qty: 4.0	 Price: 0.0
0 Item: X_110	 Qty: 1.0	 Price: 0.0
0 Item: X_129	 Qty: 1.0	 Price: 0.0
0 Item: X_115	 Qty: 1.0	 Price: 0.0
0 Item: X_121	 Qty: 1.0	 Price: 0.0
0 Item: X_124	 Qty: 1.0	 Price: 0.0
0 Item: X_133	 Qty: 1.0	 Price: 0.0


In [ ]:
for fm in map(lambda x: {'name': db.get('features', x['rootFeatureId']).first()['name'], 
                         'rootFeatureId': x['rootFeatureId'] }, 
              db.all('featuremodels')):
    print(fm)

In [ ]:
list(db.query('categories').take(5))

In [ ]:
db.query('features').take(5).take(1)